# 原始用法(需要等待较长时间搜索和函数体回传)

In [1]:
from __future__ import print_function
import fibre
import ref_tool
# logger verbose=True
logger = fibre.utils.Logger(verbose=True)
# Find an REF-Unit that is connected on the serial port /dev/ttyUSB0
# my_drive = ref_tool.find_any("serial:/dev/ttyUSB0")
my_drive = ref_tool.find_any(serial_number="3950366E3233", logger=logger)


USB discover loop
ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131

Kernel Driver was not attached
EndpointAddress for writing 3
EndpointAddress for reading 131
Connecting to device on USB device bus 1 device 1
JSON: [
{"name":"","id":0,"type":"json","access":"r"},
{"name":"serial_number","id":1,"type":"uint64","access":"r"},
{"name":"get_temperature","id":2,"type":"function","inputs":[],"outputs":[
{"name":"result","id":3,"type":"float","access":"rw"}]},
{"name":"get_voltage","id":4,"type":"function","inputs":[],"outputs":[
{"name":"result","id":5,"type":"float","access":"rw"}]},
{"name":"robot","type":"object","members":[
{"name":"calibrate_home_offset","id":6,"type":"function","inputs":[],"outputs":[]},
{"name":"homing","id":7,"type":"function","inputs":[],"outputs":[]},
{"name":"resting","id":8,"type":"function","inputs":[],"outputs":[]},
{"na

In [3]:
my_drive

serial_number = 3950366E3233 (int)
get_temperature()
get_voltage()
robot:
  calibrate_home_offset()
  homing()
  resting()
  joint_1: ...
  joint_2: ...
  joint_3: ...
  joint_4: ...
  joint_5: ...
  joint_6: ...
  joint_all: ...
  hand: ...
  reboot()
  set_enable(enable: bool)
  set_rgb_enable(enable: bool)
  set_rgb_mode(mode: int)
  move_j(j1: float, j2: float, j3: float, j4: float, j5: float, j6: float)
  move_l(x: float, y: float, z: float, a: float, b: float, c: float)
  set_joint_speed(speed: float)
  set_joint_acc(acc: float)
  set_command_mode(mode: int)
  tuning: ...

In [3]:
my_drive

serial_number = 3950366E3233 (int)
get_temperature()
get_voltage()
robot:
  calibrate_home_offset()
  homing()
  resting()
  joint_1: ...
  joint_2: ...
  joint_3: ...
  joint_4: ...
  joint_5: ...
  joint_6: ...
  joint_all: ...
  hand: ...
  reboot()
  set_enable(enable: bool)
  set_rgb_enable(enable: bool)
  set_rgb_mode(mode: int)
  move_j(j1: float, j2: float, j3: float, j4: float, j5: float, j6: float)
  move_l(x: float, y: float, z: float, a: float, b: float, c: float)
  set_joint_speed(speed: float)
  set_joint_acc(acc: float)
  set_command_mode(mode: int)
  tuning: ...

# 预先搜索和函数体缓存

In [9]:
import usb
devices = usb.core.find(find_all=True, idVendor=0x1209,idProduct=0x0d32)
count = 0
sn_to_device_dict = {}
for device in devices:
    sn_to_device_dict[device.serial_number] = device

In [12]:
sn_to_device_dict

{'3950366E3233': <DEVICE ID 1209:0d32 on Bus 001 Address 001>,
 '395D36713233': <DEVICE ID 1209:0d32 on Bus 000 Address 001>}

In [8]:
import usb
import json
import fibre.protocol
from fibre.usbbulk_transport import USBBulkTransport
from fibre.utils import Logger

# Create logger
logger = Logger(verbose=False)

# Find all devices with specified vendor and product IDs
devices = usb.core.find(find_all=True, idVendor=0x1209, idProduct=0x0d32)

# Create dictionary to store devices by serial number
sn_to_device_dict = {}
for device in devices:
    sn_to_device_dict[device.serial_number] = device
print(sn_to_device_dict)
# Store json data for each device
sn_to_json_data = {}

# Get json data for each device
for serial_number, device in sn_to_device_dict.items():
    # Reset and configure device
    device.reset()
    device.set_configuration()
    
    # Create transport layer
    bulk_device = USBBulkTransport(device, logger)
    bulk_device.init()
    
    # Create channel
    channel = fibre.protocol.Channel(
        "USB device bus {} device {}".format(device.bus, device.address),
        bulk_device, bulk_device, None, logger
    )
    
    # Get and parse json data
    json_bytes = channel.remote_endpoint_read_buffer(0)
    json_string = json_bytes.decode("ascii")
    json_data = json.loads(json_string)
    json_data = {"name": "fibre_node", "members": json_data}
    
    # Store in dictionary
    sn_to_json_data[serial_number] = json_data


{'395D36713233': <DEVICE ID 1209:0d32 on Bus 000 Address 001>, '3950366E3233': <DEVICE ID 1209:0d32 on Bus 001 Address 001>}


In [9]:
sn_to_json_data

{'395D36713233': {'name': 'fibre_node',
  'members': [{'name': '', 'id': 0, 'type': 'json', 'access': 'r'},
   {'name': 'serial_number', 'id': 1, 'type': 'uint64', 'access': 'r'},
   {'name': 'get_temperature',
    'id': 2,
    'type': 'function',
    'inputs': [],
    'outputs': [{'name': 'result', 'id': 3, 'type': 'float', 'access': 'rw'}]},
   {'name': 'get_voltage',
    'id': 4,
    'type': 'function',
    'inputs': [],
    'outputs': [{'name': 'result', 'id': 5, 'type': 'float', 'access': 'rw'}]},
   {'name': 'robot',
    'type': 'object',
    'members': [{'name': 'calibrate_home_offset',
      'id': 6,
      'type': 'function',
      'inputs': [],
      'outputs': []},
     {'name': 'homing',
      'id': 7,
      'type': 'function',
      'inputs': [],
      'outputs': []},
     {'name': 'resting',
      'id': 8,
      'type': 'function',
      'inputs': [],
      'outputs': []},
     {'name': 'joint_1',
      'type': 'object',
      'members': [{'name': 'angle', 'id': 9, 'type':

In [10]:
# Save to file
with open('sn_to_json_data.json', 'w') as f:
    json.dump(sn_to_json_data, f)

# 根据预先缓存信息加速设备驱动加载

In [23]:
import json
import fibre
logger = fibre.utils.Logger(verbose=False)
# Load the cached json data
with open('sn_to_json_data.json', 'r') as f:
    cached_json_data = json.load(f)

# Create a function to load device driver with cached json data
def load_channel_with_cache(serial_number, json_data):
    """
    Load device driver using pre-cached JSON data to accelerate initialization
    """
    import usb
    import fibre.protocol
    from fibre.usbbulk_transport import USBBulkTransport
    from fibre.utils import Logger
    
    # Create logger
    logger = Logger(verbose=False)
    
    # Find device with specified serial number
    device = usb.core.find(idVendor=0x1209, idProduct=0x0d32, 
                          serial_number=serial_number)
    
    if not device:
        raise Exception(f"Device with serial number {serial_number} not found")
        
    # Reset and configure device
    device.reset()
    device.set_configuration()
    
    # Create transport layer
    bulk_device = USBBulkTransport(device, logger)
    bulk_device.init()
    
    # Create channel with cached json data
    channel = fibre.protocol.Channel(
        "USB device bus {} device {}".format(device.bus, device.address),
        bulk_device, bulk_device, None, logger
    )
    
    # Use cached json data instead of reading from device
    channel._interface_definition_crc = json_data.get('json_crc', 0)
    channel._remote_objects = json_data
    
    return channel

In [24]:
# Load device driver using cache
serial_number = "3950366E3233"  
json_data = cached_json_data[serial_number]
channel = load_channel_with_cache(serial_number, json_data)


In [39]:

obj = fibre.remote_object.RemoteObject(json_data, None, channel, logger)
json_bytes = json.dumps(json_data['members']).encode('utf-8')
json_crc16 = fibre.protocol.calc_crc16(fibre.protocol.PROTOCOL_VERSION, json_bytes)
obj.__dict__['_json_data'] = json_data['members']
obj.__dict__['_json_crc'] = json_crc16